### How to discover the problem


We know that this is a text file from the txt file ending.

The first thing I did was open the file in my favourite editor, vim, to see what the error might be. As soon as I see the weird characters, I know I've found the issue:


![](imgs/problem_file_in_vim.png)


I google "vim ^L" and discover it's the form feed character. That shouldn't be too much of an issue. The umlaut on the U might be though. 

I decide to open the file in a notepad style text editor and I see the same. Maybe the problem is that [Ü](https://en.wikipedia.org/wiki/%C3%9C) can belong to a number of charsets - but they all have a form feed char.

So lets break out hexdump which will show us what is actually written in the file in a hexidecimal representation of the underlying binary.

![](imgs/hexdump.png)

Here we see the Ü represented as a `�` and `o.o.` - but we can also see that the hex is `6f dc 6f 0d`

```
00000a70  20 20 20 20 20 20 20 20  20 20 20 20 6f dc 6f 0d  |            o.o.|
0000a70                                                   o   �   o  \r
```

Going back to the wikipedia page for Ü we can see that there are a number of charsets that this could belong to when wetake into account the representation of DC (EBCDIC family, ISO 8859-1/3/4/9/10/14/15/16) and that this could take time.

Most importantly, these characters are OCR *artefacts* not data. So we can just not read them or chop them off. Which is what we do in this line 

`text = file_handle.read(2650).decode('ascii')` 

This says "read the first 2650 bits of the file, treat them as ASCII".

How did I know to do 2650? The left hand column of my hexdump is the "offset" - how far into the file hexdump is - in hexidecimal. So I just grabbed the address *after* the last bit of information (`00000a60`) and [converted it to the decimal value](https://www.mathwarehouse.com/solved-problems/conversions/convert-a60-from-hexadecimal-to-decimal) that the Python `read` command required and rounded down because human brain.

This is a hack because despite all our computing power, in the end we needed to make the decision that this wasn't data ourselves - we needed to use our eyeballs, our understanding of our end goal and some non python coding. This is justifiable, but you need to justify, not just expect readers to understand the magic.